In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [6]:
#载入数据集
mnist = input_data.read_data_sets('MNIST_data',one_hot = True)#当前路径，‘’可以选择存放路径

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [7]:
#每个批次大小
batch_size = 100
#计算一共有多少个批次
#数据集的数量整除批次大小=多少个批次
n_batch = mnist.train.num_examples // batch_size

#定义两个placeholder
x = tf.placeholder(tf.float32,[None,784])#把批次传进去，开始是100，最后一批不一定是多少，我们把平面28*28做成向量传入
y = tf.placeholder(tf.float32,[None,10])#十个标签
keep_prob = tf.placeholder(tf.float32)

#创建一个简单的神经网络
#只用到两个层，784个神经元，输出层为10个神经元
W1=tf.Variable(tf.truncated_normal([784,2000],stddev=0.1))
b1=tf.Variable(tf.zeros([2000])+0.1)
# W = tf.Variable(tf.zeros([784,10]))
# b = tf.Variable(tf.zeros([10]))
L1 = tf.nn.tanh(tf.matmul(x,W1)+b1)
L1_drop = tf.nn.dropout(L1,keep_prob)#L1为某层的输出，keep_prob=1为所有神经元工作


W2=tf.Variable(tf.truncated_normal([2000,2000],stddev=0.1))
b2=tf.Variable(tf.zeros([2000])+0.1)
L2 = tf.nn.tanh(tf.matmul(L1_drop,W2)+b2)
L2_drop = tf.nn.dropout(L2,keep_prob)#L1为某层的输出，keep_prob=1为所有神经元工作

W3=tf.Variable(tf.truncated_normal([2000,1000],stddev=0.1))
b3=tf.Variable(tf.zeros([1000])+0.1)
L3 = tf.nn.tanh(tf.matmul(L2_drop,W3)+b3)
L3_drop = tf.nn.dropout(L3,keep_prob)#L1为某层的输出，keep_prob=1为所有神经元工作

W4=tf.Variable(tf.truncated_normal([1000,10],stddev=0.1))
b4=tf.Variable(tf.zeros([10])+0.1)



prediction = tf.nn.softmax(tf.matmul(L3_drop,W4)+b4)

#定义代价函数
#loss = tf.reduce_mean(tf.square(y-prediction))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y,logits = prediction))#填写标签和prediction
#使用梯度下降法
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

#定义一个求准确率的方法
#如果两个相同就会返回True,不相同就返回False，然后存入correct_prediction
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大值所在的位置

#求准确率
#首先把bool值转化为32为浮点值，然后求平均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [11]:
mnist.train.images[0].shape

(784,)

In [8]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(21):#把所有的图片训练21次
        for batch in range (n_batch):#一批一批的迭代，一共运行n_batch次
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)#每次获得100张图片
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
        
        test_acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        train_acc = sess.run(accuracy,feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_prob:1.0})
        print('Iter'+str(epoch) + ',Testing accuracy'+ str(test_acc)+'training accuracy'+str(train_acc))
            

Iter0,Testing accuracy0.9417training accuracy0.9503091
Iter1,Testing accuracy0.9525training accuracy0.9685091
Iter2,Testing accuracy0.9556training accuracy0.9764909
Iter3,Testing accuracy0.9594training accuracy0.9808364
Iter4,Testing accuracy0.9602training accuracy0.98358184


KeyboardInterrupt: 